In [85]:
"""
    Maps Standings and Fixtures for building the Training Data
"""

'\n    Maps Standings and Fixtures for building the Training Data\n'

In [108]:
import json
import os
import pandas as pd
from pandas import json_normalize

In [109]:
# Historicity: multi season directory
year_window = 1
start_season = 2017
seasons = list(range(start_season, start_season - year_window, -1))
print(seasons)

[2017]


In [110]:
# Multi League directory
country = 'ES'
dirCountry = '../' + country + '/'

In [111]:
league = '140'
season = '2017'

In [112]:
# League directory
dirName = dirCountry + league + '/'

In [113]:
# The last week played
curr_week = 38

In [114]:
df = pd.DataFrame()
standings = None
curr_standings = None
away_team_stands = None
fixture = None
# This will prevent duplicated fixtures
visited_fixtures = []
# This will help to map to standings to week: team : stands
visited_standings = {}
"""visited_standings['1'] = {}"""
# Easy access to last standings
current_standings = {}

# Principal leagues are sorted ascendent
dirLeague = sorted(os.listdir(dirCountry))

for league in dirLeague:
    if league == ".DS_Store":
        continue
    if "csv" in league:
        continue
    for season in seasons:
        dirName = dirCountry + league + '/' + str(season) + '/' 
        for subdir in os.listdir(dirName):
            if subdir == ".DS_Store":
                continue
            if subdir == "teams.json":
                continue
            if ".csv" in subdir:
                continue
            team_id = subdir
            # Entering directory of a team
            dirTeam = dirName + team_id + '/'
            
            # Initializes all first standings
            """
            if team_id not in visited_standings['1']:
                json_standings = dirTeam + team_id + '_' + str(1) + '.json'
                with open(json_standings) as f:
                    first_standings = json.load(f)
                    visited_standings['1'][team_id] = first_standings
            """
            
            # Initializes all current standings
            if team_id not in current_standings:
                json_standings = dirTeam + team_id + '_' + str(curr_week) + '.json'
                with open(json_standings) as f:
                    curr_standings = json.load(f)
                    current_standings[team_id] = curr_standings
            else:
                curr_standings = current_standings[team_id]
            
            # Gets standings from a team if they have not being added
            for week in list(range(1, 39, 1)):
                week = str(week)
                if week not in visited_standings:
                    visited_standings[week] = {}
                    if team_id not in visited_standings[week]:
                        try:
                            json_standings = dirTeam + team_id + '_' + week + '.json'
                            with open(json_standings) as f:
                                standings = json.load(f)
                                visited_standings[week][team_id] = standings
                        except:
                            visited_standings[week][team_id] = curr_standings
                            standings = curr_standings

            # Directory of team in home
            dirHome = dirTeam + 'home/'
            for file in os.listdir(dirHome):
                # Check if fixture hasn't being added to dataframe
                if file not in visited_fixtures:
                    path = dirHome + file
                    with open(path) as f:
                        fixture = json.load(f)
                        # Adds id of fixture as "id_fixture.json"
                        visited_fixtures.append(file)
                        # Next, get away team's external info
                        away_team = str(fixture["team_away"]["id"])
                        w = fixture["week"]
                        w = ''.join(filter(str.isdigit, w))
                        # Gets standings from away_team
                        """if w == '1':
                            try:
                                fileAwayTeam = dirName + away_team + '/' + away_team + '_' + str(1) + '.json'
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    visited_standings['1'][away_team] = away_team_stands
                            except:
                                print(fileAwayTeam)
                                print("Something rare")
                        """
                        if away_team in visited_standings[w]:
                            away_team_stands = visited_standings[w][away_team]
                        else:
                            try:
                                fileAwayTeam = dirName + away_team + '/' + away_team + '_' + week + '.json'
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    visited_standings[w][away_team] = away_team_stands
                            except:
                                fileAwayTeam = dirName + away_team + '/' + away_team + '_' + str(curr_week) + '.json'
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    current_standings[away_team] = away_team_stands
                        # Home team standings
                        if team_id in visited_standings[w]:
                            standings = visited_standings[w][team_id]
                        else:
                            standings = current_standings[team_id]
                        # Gets away current standings if fixtures is set to null
                        if fixture["goals_away"] is None:
                            away_team_stands = current_standings[away_team]
                            # Changes if game to be played
                            standings = current_standings[team_id]
                        else:
                            if away_team in visited_standings[w]:
                                away_team_stands = visited_standings[w][away_team]
                            else:
                                away_team_stands = current_standings[away_team]
                    # Three main jsons to a dataframe
                    # standings "this is the home team"
                    # away_team_stands
                    # the fixture itself
                    # The premise that away standings from home team aren't relevant
                    try:
                        #del standings['stats_away']
                        fixture.update({"team_home": standings})
                    except:
                        print("Couldn't delete stats away")
                    # The premise that home standings from away team aren't relevant
                    try:
                        #del away_team_stands['stats_home']
                        fixture.update({"team_away": away_team_stands})
                    except:
                        print("Couldn't delete stats home")
                    # Adding league
                    fixture.update({"league": int(league)})
                    # Adding season
                    fixture.update({"season": season})
                    df = df.append(json_normalize(fixture))
                else:
                    continue

In [115]:
df.head(10)

,id,season,week,goals_home,goals_away,league,team_home.team.id,team_home.team.name,team_home.rank,team_home.points,...,stats_away.fouls,stats_away.corners,stats_away.offside,stats_away.possession,stats_away.c_yellow,stats_away.c_red,stats_away.saves,stats_away.p_total,stats_away.p_accurate,stats_away.p_percentage
0,9545,2017,35,4,1,140,533,Villarreal,5,37,...,8,7,2,63%,None,None,3,615,540,88%
0,9642,2017,25,1,0,140,533,Villarreal,5,37,...,21,8,5,47%,3,None,3,357,245,69%
0,9739,2017,15,0,2,140,533,Villarreal,5,37,...,13,4,5,68%,1,None,5,778,693,89%
0,9667,2017,23,1,2,140,533,Villarreal,5,37,...,15,2,2,30%,4,None,5,245,142,58%
0,9695,2017,20,2,1,140,533,Villarreal,5,37,...,16,4,None,42%,2,None,4,312,226,72%
0,9684,2017,21,4,2,140,533,Villarreal,5,37,...,6,9,1,63%,1,None,3,587,506,86%
0,9627,2017,27,0,2,140,533,Villarreal,5,37,...,18,2,3,38%,None,None,5,292,189,65%
0,9512,2017,38,2,2,140,533,Villarreal,5,37,...,10,5,3,62%,3,None,2,670,618,92%
0,9859,2017,3,3,1,140,533,Villarreal,5,37,...,9,3,1,53%,None,None,4,494,423,86%
0,9760,2017,13,2,3,140,533,Villarreal,5,37,...,17,6,4,54%,3,None,2,522,444,85%


In [116]:
df.shape

(380, 122)

In [117]:
# csv to save
file_title = str(season) + '_' + str(curr_week) + '.csv'
csv_file = dirName + file_title
print(file_title)
print(csv_file)

2017_38.csv
../ES/140/2017/2017_38.csv


In [118]:
df.to_csv(csv_file)